## Mask the weighted degree centrality (wDC) maps using the two masks from Holiga et al., 2019 (https://www.science.org/doi/10.1126/scitranslmed.aat9223) and calculate the mask-averaged wDC values

In [1]:
import os
import glob

import numpy as np
import pandas as pd

import nibabel as nib
from nilearn.maskers import NiftiMasker

In [2]:
# Load an image from a single brain:
def img_load(folder, img):
    img_path = os.path.join(folder, img)
    fmri_image = nib.load(img_path)
    #print(fmri_image.shape)
    return fmri_image

In [3]:
# Load a resampled, binarised GM mask
def mask_img_load(folder, img):
    img_path = os.path.join(folder, img)
    mask = nib.load(img_path)
    return mask

In [4]:
# Extract the timeseries from a single session
def extract_time_series(fmri_image, mask):
    brain_masker = NiftiMasker(mask, memory='nilearn_cache', memory_level=1, verbose=0)
    brain_time_series = brain_masker.fit_transform(fmri_image)
    return brain_time_series, brain_masker

In [5]:
# Transform the FC or DC array back into a NIfTI volume and save
def array_to_nifti(brain_masker,array):
    img = brain_masker.inverse_transform(array.T)
    return img

In [6]:
def mask_average_wDC(rootDIR, SUBlist, ROImask):
    dc_list = []
    for i in range(len(SUBlist)):
        
        # load an image from a single brain:
        dc_image = img_load(rootDIR, SUBlist[i])
        print("Participant " + SUBlist[i])
        #print("The image dimensions are: " + str(dc_image.shape))
        
        # load the ROI mask
        roi_mask = mask_img_load(rootDIR, ROImask)
        #print("ROI mask loaded.")
        
        # create a NiftiMasker object and extract the DC
        dc_vals, brain_masker = extract_time_series(dc_image, roi_mask)
        #print("Time series extracted.")
        
        # check for missing values
        n_zeros = np.count_nonzero(dc_vals==0)
        print(f'The number of missing values is {n_zeros}')
        
        # calculate mean DC
        mean_dc = np.mean(dc_vals)
        
        # add to the list of images
        dc_list.append(mean_dc)
        print("Image added to the list.")
    
    print("Done!")
    return dc_list

In [7]:
# Generate the lists of ID's

In [ ]:
os.chdir('/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Data/Covariates')
covariate_sheet = pd.read_excel('Covariates.xlsx', sheet_name=0, engine='openpyxl')
covariate_sheet

In [ ]:
id_list = [covariate_sheet.iloc[:,0][i] for i in range(covariate_sheet.iloc[:,0].shape[0])]
id_list 

In [10]:
placebo_list = []
cbd_list = []

In [11]:
# Split the sessions into the 3 respective lists from above
for i in range(covariate_sheet.shape[0]):
    if covariate_sheet.iloc[i]["Placebo"]:
        placebo_list.append(covariate_sheet.iloc[i]["ID"])
    elif covariate_sheet.iloc[i]["CBD"]:
        cbd_list.append(covariate_sheet.iloc[i]["ID"])
    else:
        print("Something's wrong here..")

In [12]:
#placebo_list

In [13]:
placebo_td = []
placebo_autism = []
cbd_td = []
cbd_autism = []

In [14]:
# Split each of those into 2 new lists, of TD and ASC sessions, respectively (see just above)
for i in range(len(placebo_list)):
    if int(placebo_list[i][0]) == 0:
        placebo_td.append(placebo_list[i])
    elif int(placebo_list[i][0]) == 1:
        placebo_autism.append(placebo_list[i])
    else:
        print("Something's wrong here..")
        
for i in range(len(cbd_list)):
    if int(cbd_list[i][0]) == 0:
        cbd_td.append(cbd_list[i])
    elif int(cbd_list[i][0]) == 1:
        cbd_autism.append(cbd_list[i])
    else:
        print("Something's wrong here..")

In [15]:
###################################################################################################################

In [16]:
# Choose whether to use the wDC or the DC folder..
work_dir = '/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Static_FC/DC_data/wDC_Z'

In [ ]:
###################################################################################################################

In [35]:
# Choose the set of ID's
name_beginning = 'MAGACAN'

ses_id = cbd_autism

name_ending = '_wDC_Z.nii'

In [ ]:
# Create the session list
ses_list = []
for i in range(len(ses_id)):
    session_filename = name_beginning + ses_id[i] + name_ending
    ses_list.append(session_filename)
ses_list

In [37]:
# ses_list = img_list

In [ ]:
# Run program
dc_list = mask_average_wDC(work_dir, ses_list, 'final_asc_td_mask.nii.gz')

In [ ]:
# Convert the list to a Pandas DataFrame
dc_df = pd.DataFrame(dc_list, columns = ['CBD'])
dc_df = dc_df.set_index([ses_id])
dc_df

In [43]:
# Save to disk (as .csv)
os.chdir('/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Static_FC/Analysis/Step_2_Mean_wDC')
dc_df.to_csv('ASC_CBD_wDC_Z_hyper.csv'
             #, index=False
            )